In [1]:
import collections
#todo: divide the terms into ohters, interactionType, detectionMethod, Interaction Source

In [20]:
"""
June 10, 2020
Parsing Steps:
1. build the tree by the psi-mi number. A dictionary {psi-mi: node} is used 
to access nodes as well.
2. save all the tree nodes in the subtree of the three nodes into three set:
    id: MI:0001 name: interaction detection method
    id: MI:0190 name: interaction type
    id: MI:0444 name: database citation
3. save the nodes in the three sets to the corresponding enumearation schema
"""

'\nJune 10, 2020\nParsing Steps:\n1. build the tree by the psi-mi number\n2. save all the tree nodes in the subtree of the three nodes into three set:\n    id: MI:0001 name: interaction detection method\n    id: MI:0190 name: interaction type\n    id: MI:0444 name: database citation\n3. save the nodes in the three sets to the corresponding enumearation schema\n'

In [21]:
with open('mi.owl','r') as fp:
    file = fp.read()

In [22]:
#get the file instruction and save into a dictionary
fileHeader = file.split('\n\n')[0]

In [4]:
fileHeaderDic = collections.defaultdict(list)
pairs = fileHeader.split("\n")
for pair in pairs:
    pairList = pair.split(":")
    key = pairList[0]
    value = ":".join(pairList[1:])
    fileHeaderDic[key].append(value)

In [5]:
fileHeaderDic.keys()

dict_keys(['format-version', 'date', 'saved-by', 'auto-generated-by', 'subsetdef', 'synonymtypedef', 'default-namespace', 'remark', 'ontology'])

In [6]:
#In this version synonym is saved as a string and is not linked to each synonym type.

In [7]:
#Three subsets are saved to enumeration schema 
schemaPiece = """Node: dcid:BiomedicalOntologySubsetEnum
typeOf: schema:Enumeration
name: "BiomedicalOntologySubsetEnum"
description: \"The subset enumeration in biomedical ontologies\""""
schemaEnum = [schemaPiece]

for subsetPair in fileHeaderDic['subsetdef']:
    subsetList = subsetPair.split()  
    dcid = subsetList[0]
    description = ' '.join(subsetList[1:])
    schemaPiece = 'Node: dcid:' + dcid + '\n'+\
'typeOf: dcs:BiomedicalOntologySubsetEnum\nname: “'+dcid+'”\ndescription: '+\
description
    schemaEnum.append(schemaPiece)
schemaEnumText = '\n\n'.join(schemaEnum)

In [ ]:
class Node(Object):
    def __init__(self, value):
        self.id = value
        # one node can have multiple parent nodes
        self.parent = []
        # one node can have multiple child nodes
        self.child = []
id2node = {}
# build nodes and create the id2node dictionary at first iteration
for termText in fileTerm:
    if not termText.startswith("[Term]"):
        continue
    # idString example: "MI:0000"
    idString = termText.split("\n")[1].split(" ")[1]
    id2node[idString] = Node(idString)

for termText in fileTerm:
    
    
    
# build the parent-child relation



In [24]:
fileTerms[0].split("\n")

['[Term]',
 'id: MI:0000',
 'name: molecular interaction',
 'def: "Controlled vocabularies originally created for protein protein interactions, extended to other molecules interactions." [PMID:14755292]',
 'subset: Drugable',
 'subset: PSI-MI_slim',
 'synonym: "mi" EXACT PSI-MI-short []']

In [8]:
with open('BioOntologySubsetEnumSchema.mcf','w') as fp:
    fp.write(schemaEnumText)

In [9]:
fileTerms = file.split('\n\n')[1:]

In [17]:
def getSchemaFromText(term):
    """
    Takes a list with each item containing the information, return a data schema. 
    Example:
    term = ['id: MI:0000',
     'name: molecular interaction',
     'def: "Controlled vocabularies originally created for protein protein interactions, extended to other molecules interactions." [PMID:14755292]',
     'subset: Drugable',
     'subset: PSI-MI_slim']
    """
    termDic = collections.defaultdict(list)
    for line in term:
        lineList = line.split(": ")
        key = lineList[0]
        value = ": ".join(lineList[1:])
        termDic[key].append(value)
    try:
        defLong = termDic['def'][0]
    except:
        print("No def attribute",term)
    try:
        idStart = defLong.rfind('[')
        description = defLong[:idStart]
        termDic['def'] = [description]
        IDList = defLong[idStart+1:-1].split(", ")
        termDic['pubID']= IDList
        
    except:
        print(defLong)
    
    schemaPieceList = []
    keyList = ["name","def","pubID","subset","synonym","is_obsolete", "is_a","relationship"]
    key = "id"
    curLine = "Node: dcid:bio/psimi_" + "".join(termDic[key][0].split(':'))
    schemaPieceList.append(curLine)
    curLine = "typeOf: dcs:MolecularInteractionOntology"
    schemaPieceList.append(curLine)
    
    termDic['def'] = ["\""+ termDic['name'][0][0].upper() + termDic['name'][0][1:]  + ": " + termDic['def'][0][1:]]
    termDic['name'] = ["psimi_" + "".join(termDic[key][0].split(':'))]

    for key in keyList:

        if key=="name" and key in termDic:
            itemList = []
            for i in range(len(termDic[key])):
                itemList.append("\"" + termDic[key][i] + "\"")
            curLine = "name: " + ",".join(itemList)
            schemaPieceList.append(curLine)
            
        elif key=="def" and key in termDic:                
            itemList = []
            for i in range(len(termDic[key])):
                itemList.append(termDic[key][i])
            curLine = "description: " + ",".join(itemList)
            schemaPieceList.append(curLine)
            
        elif key=="pubID" and key in termDic:
            itemList = []
            for i in range(len(termDic[key])):
                itemList.append( "\"" + termDic[key][i] + "\"")
            curLine = "pubID: " +  ",".join(itemList)
            schemaPieceList.append(curLine)
            
        elif key=="subset" and key in termDic:
            itemList = []
            for i in range(len(termDic[key])):
                itemList.append(termDic[key][i])
            curLine = "subsetOf: dcs:"  + ",".join(itemList)
            schemaPieceList.append(curLine)
            
        elif key=="synonym" and key in termDic:
            itemList = []
            for i in range(len(termDic[key])):
                itemList.append("\"" + termDic[key][i] + "\"" )
            curLine = "synonym: " + ",".join(itemList)
            schemaPieceList.append(curLine)
                
        elif key=="is_obsolete" and key in termDic:
            itemList = []
            for i in range(len(termDic[key])):
                itemList.append( termDic[key][i]  )
            curLine = "isObsolete: " + ",".join(itemList)
            schemaPieceList.append(curLine)
            
        elif key=="is_a" and key in termDic:
            itemList = []
            for i in range(len(termDic[key])):
                itemList.append('dcid/bio/psimi_' + "".join(termDic[key][i].split(" !")[0].split(":")))
            curLine = "subClassOf: " + ",".join(itemList)
            schemaPieceList.append(curLine)
                
        elif key=="relationship" and key in termDic:
            itemList = []
            for i in range(len(termDic[key])):
                itemList.append('dcid/bio/psimi_' + "".join(termDic[key][i].split(" ")[1].split(":")))
            curLine = "psimiPartOf: " + ",".join(itemList)
            schemaPieceList.append(curLine)
    return "\n".join(schemaPieceList)

In [18]:
schemaList = []
schema = '''Node: dcid:MolecularInteractionOntology
typeOf: schema:Class
subClassOf: Thing
description: \"A structured controlled vocabulary for the annotation of experiments concerned with protein-protein interactions.\"'''
schemaList.append(schema)
# count = 0
for termText in fileTerms:
    if not termText.startswith("[Term]"):
        continue
    term = termText.split("\n")[1:]
    schema = getSchemaFromText(term)
    schemaList.append(schema)
#     count+=1
#     if count>5:
#         break
schemaEnumText = "\n\n".join(schemaList)
with open('BioOntologySchema.mcf','w') as fp:
    fp.write(schemaEnumText)

    
    